In [3]:
import opendatasets as od
import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import make_column_transformer
# from sklearn.pipeline import make_pipeline
# from sklearn.metrics import classification_report
# from sklearn.metrics import r2_score
import sklearn
import joblib
import numpy as np

In [ ]:
od.download(
    "https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction")

In [2]:
df_train = pd.read_csv("heart-failure-prediction/heart.csv")

In [3]:
df_train

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [4]:
X = df_train.loc[:, df_train.columns != 'HeartDisease']
y = df_train["HeartDisease"]

In [5]:
X

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up
...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat


In [6]:
y

0      0
1      1
2      0
3      1
4      0
      ..
913    1
914    1
915    1
916    1
917    0
Name: HeartDisease, Length: 918, dtype: int64

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=1)

In [8]:
ohe=OneHotEncoder()
ohe.fit(X[['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']])

OneHotEncoder()

In [9]:
column_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']),
                                remainder='passthrough')

In [10]:
rf = RandomForestClassifier(random_state=42, n_jobs=-1, max_depth=8,
                                       n_estimators=100, oob_score=True)

In [11]:
pipe = make_pipeline(column_trans,rf)

In [12]:
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['F', 'M'], dtype=object),
                                                                            array(['ASY', 'ATA', 'NAP', 'TA'], dtype=object),
                                                                            array(['LVH', 'Normal', 'ST'], dtype=object),
                                                                            array(['N', 'Y'], dtype=object),
                                                                            array(['Down', 'Flat', 'Up'], dtype=object)]),
                                                  ['Sex', 'ChestPainType',
                                                   'RestingECG',
                                                   'ExerciseAngina',
                                                   'ST_Slope'])])),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=8, n_jobs=-1, oob_score=True,
                                        random_state=42))])

In [13]:
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)

In [14]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.85      0.86       153
           1       0.89      0.91      0.90       215

    accuracy                           0.88       368
   macro avg       0.88      0.88      0.88       368
weighted avg       0.88      0.88      0.88       368



In [29]:
pipe.predict(pd.DataFrame(columns=X_test.columns,data=np.array([49,'F','NAP',160,180,0,'Normal',156,'N',1,'Flat']).reshape(1,11)))

array([1], dtype=int64)

In [28]:
joblib.dump(pipe,open('HeartFailurePrediction.pkl','wb'))

In [27]:
pd.DataFrame(columns=['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
                               'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope'],data=np.array([49,'F','NAP',160,180,0,'Normal',156,'N',1,'Flat']).reshape(1,11))

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,49,F,NAP,160,180,0,Normal,156,N,1,Flat


In [45]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB
